<a href="https://colab.research.google.com/github/lrfanton/lrfEndowment/blob/main/LRFEndowment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ============
# Parametrar
# ============
SEED = 42
rng = np.random.default_rng(SEED)

START_CAPITAL_MSEK = 7_000   # 7 BSEK
YEARS = 10
N_SIMS = 10_000

ALPHA = 1          # smoothing (hybridregel)
INFLATION = 0.02      # årlig inflation
SPENDING_RATE = 0.05  # årligt spending rate

# --- Portfölj ---
PORTFOLIO_NAME = "7% portfölj (balanserad)"
MU_ANNUAL = 0.07
SIGMA_ANNUAL = 0.12

# Konvertera till månadsparametrar
MU_MONTH = (1 + MU_ANNUAL) ** (1/12) - 1
SIGMA_MONTH = SIGMA_ANNUAL / np.sqrt(12)

# ===========================
# Simuleringsfunktion (sparar årsvis kapital)
# ===========================
def simulate_paths(mu_month, sigma_month, years, n_sims, start_cap_msek, spending_rate, alpha, inflation, rng):
    paths = np.empty((n_sims, years+1), dtype=float)  # inkludera år 0
    for i in range(n_sims):
        cap = float(start_cap_msek)
        payout_annual = spending_rate * cap
        paths[i, 0] = cap

        for year in range(1, years+1):
            monthly_payout = payout_annual / 12
            for _ in range(12):
                ret = rng.normal(mu_month, sigma_month)
                cap = cap * (1 + ret) - monthly_payout
            paths[i, year] = cap
            # uppdatera hybridregel årsvis
            target = spending_rate * cap
            payout_annual = alpha * payout_annual * (1 + inflation) + (1 - alpha) * target

    return paths

# ==================
# Kör simulering
# ==================
paths = simulate_paths(
    mu_month=MU_MONTH, sigma_month=SIGMA_MONTH, years=YEARS, n_sims=N_SIMS,
    start_cap_msek=START_CAPITAL_MSEK, spending_rate=SPENDING_RATE,
    alpha=ALPHA, inflation=INFLATION, rng=rng
)

# ==================
# Sammanställning
# ==================
years = np.arange(0, YEARS+1)
median = np.median(paths, axis=0)
p10 = np.percentile(paths, 10, axis=0)
p90 = np.percentile(paths, 90, axis=0)

# ==================
# Plotly – konfidensintervall
# ==================
fig = go.Figure()

# Skuggat område P10–P90
fig.add_traces([
    go.Scatter(x=years, y=p90, line=dict(width=0), showlegend=False),
    go.Scatter(
        x=np.concatenate([years, years[::-1]]),
        y=np.concatenate([p90, p10[::-1]]),
        fill="toself", fillcolor="rgba(0,100,200,0.2)", line=dict(width=0),
        showlegend=True, name="P10–P90"
    ),
    go.Scatter(x=years, y=median, mode="lines", line=dict(color="blue"),
               name="Median")
])

fig.update_layout(
    title=f"{PORTFOLIO_NAME} – Kapital vid årsskiften (10 000 simuleringar)",
    xaxis_title="År",
    yaxis_title="Kapital (MSEK)",
    height=500
)

# Röd linje = startkapital
fig.add_hline(y=START_CAPITAL_MSEK, line=dict(color="red", dash="dash"))

fig.show()

In [ ]:
finals

In [ ]:
import numpy as np
import pandas as pd

# ============
# Parametrar (samma som innan)
# ============
SEED = 123
rng = np.random.default_rng(SEED)

START_CAPITAL_MSEK = 7_000
YEARS = 10

ALPHA = 0.80
INFLATION = 0.02
SPENDING_RATE = 0.05

MU_ANNUAL = 0.07
SIGMA_ANNUAL = 0.12

MU_MONTH = (1 + MU_ANNUAL) ** (1/12) - 1
SIGMA_MONTH = SIGMA_ANNUAL / np.sqrt(12)

# ============
# En simulering i detalj
# ============
def simulate_one(mu_month, sigma_month, years, start_cap_msek, spending_rate, alpha, inflation, rng):
    cap = float(start_cap_msek)
    payout_annual = spending_rate * cap
    records = []

    for year in range(1, years+1):
        monthly_payout = payout_annual / 12
        start_cap = cap
        year_ret = 1.0
        total_payout = 0

        for _ in range(12):
            ret = rng.normal(mu_month, sigma_month)
            year_ret *= (1 + ret)
            cap = cap * (1 + ret) - monthly_payout
            total_payout += monthly_payout

        # Avkastning i procent (innan uttag): (slutvärde före uttag / startvärde - 1)
        gross_return = year_ret - 1

        records.append({
            "År": year,
            "Startkapital": start_cap,
            "Bruttoavkastning": gross_return,
            "Årligt uttag": total_payout,
            "Slutkapital": cap
        })

        # Uppdatera hybridregel för nästa års uttag
        target = spending_rate * cap
        payout_annual = alpha * payout_annual * (1 + inflation) + (1 - alpha) * target

    return pd.DataFrame(records)

# Kör en enskild simulering
df_one = simulate_one(
    mu_month=MU_MONTH, sigma_month=SIGMA_MONTH, years=YEARS,
    start_cap_msek=START_CAPITAL_MSEK, spending_rate=SPENDING_RATE,
    alpha=ALPHA, inflation=INFLATION, rng=rng
)

df_one

,År,Startkapital,Bruttoavkastning,Årligt uttag,Slutkapital
0,1,7000.000000,0.045761,350.000000,6971.796741
1,2,6971.796741,0.210619,355.317967,8050.554101
2,3,8050.554101,0.085468,370.445002,8373.880222
3,4,8373.880222,0.151254,386.021924,9252.980934
4,5,9252.980934,0.045472,407.523699,9271.106588
5,6,9271.106588,-0.012031,425.250405,8725.846838
6,7,8725.846838,0.242917,434.262799,10379.012235
7,8,10379.012235,0.063995,458.148566,10559.041568
8,9,10559.041568,0.128557,479.439646,11392.562575
9,10,11392.562575,0.031577,505.148377,11250.760846


In [ ]:
import plotly.graph_objects as go

# Skapa figuren
fig = go.Figure()

# Kapital (slutkapital varje år)
fig.add_trace(go.Scatter(
    x=df_one["År"], y=df_one["Slutkapital"],
    mode="lines+markers", name="Kapital (MSEK)",
    line=dict(color="blue")
))

# Årliga uttag
fig.add_trace(go.Bar(
    x=df_one["År"], y=df_one["Årligt uttag"],
    name="Uttag (MSEK)", marker_color="orange",
    yaxis="y2"   # sekundäraxis
))

# Bruttoavkastning i %
fig.add_trace(go.Scatter(
    x=df_one["År"], y=100*df_one["Bruttoavkastning"],
    mode="lines+markers", name="Bruttoavkastning (%)",
    line=dict(color="gray", dash="dot"),
    yaxis="y3"   # tredje axel
))

# Layout
fig.update_layout(
    title="Exempel på en simulering – Kapital, Uttag och Avkastning",
    xaxis=dict(title="År"),
    yaxis=dict(title="Kapital (MSEK)", side="left", showgrid=False),
    yaxis2=dict(title="Årligt uttag (MSEK)", overlaying="y", side="right", showgrid=False),
    yaxis3=dict(title="Bruttoavkastning (%)", overlaying="y", side="right", position=1.07, showgrid=False),
    barmode="group",
    height=600
)

fig.show()

ValueError: 
    Invalid value of type 'builtins.float' received for the 'position' property of layout.yaxis
        Received value: 1.07

    The 'position' property is a number and may be specified as:
      - An int or float in the interval [0, 1]